In [1]:
import pandas as pd
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [4]:
# Create the model
model = LpProblem(name='Are_You_The_One', sense=LpMinimize)

# Define decision variables
var = [[LpVariable(name='{},{}'.format(i,j), lowBound=0, cat='Binary') for j in range(10)] for i in range(10)]
var_flat = [x for tup in var for x in tup]

# Set objective function
model += lpSum(var_flat)

# Define constraints - results of each round
# In rounds: index is guys, values is girls
# In truth_booths: key is (guy, girl), value is result
rounds = []
rounds.append( {'pairs': [3, 5, 7, 8, 0, 4, 2, 1, 9, 6], 'num_matches': 2} )
rounds.append( {'pairs': [5, 7, 9, 2, 6, 8, 4, 3, 0, 1], 'num_matches': 4} )
rounds.append( {'pairs': [5, 7, 4, 6, 2, 8, 0, 1, 9, 3], 'num_matches': 2} )
rounds.append( {'pairs': [7, 1, 0, 6, 8, 2, 3, 5, 9, 4], 'num_matches': 2} )
rounds.append( {'pairs': [5, 8, 7, 2, 6, 1, 4, 3, 9, 0], 'num_matches': 5} )
truth_booths = {(0,0): 0, (6,2): 0, (4,3): 0, (8,0): 0, (5,8): 0, (8,9): 1}

# each person has at most one confirmed match (allow less than for unsure matches)
for i in range(10):
    model += (lpSum(var[i]) <= 1)
for i in range(10):
    girl_matches = [var[j][i] for j in range(10)]
    model += (lpSum(girl_matches) <= 1)
    
# truth booth results
for k, v in truth_booths.items():
    m, f = k
    model += (var[m][f] == v)
    
# matching ceremony results
for r in rounds:
    matches = [var[m][f] for m, f in zip(range(10), r['pairs'])]
    model += (lpSum(matches) == r['num_matches'])
    
# Solve the optimization problem
status = model.solve()

# Get the results
print('status: {}, {}'.format(model.status, LpStatus[model.status]))
print('objective: {}'.format(model.objective.value()))
    
guys = ('Ethan', 'Ryan', 'Wes', 'Chris S', 'Chris T', 'Dre', 'JJ', 'Adam', 'Dillan', 'Joey')
girls = ('Jessica', 'Brittany', 'Simone', 'Shanley', 'Jacy', 'Amber', 'Paige', 'Kayla', 'Ashleigh', 'Coleysia')
results_table = []
for v in var:
    results_table.append([int(v[i].value()) for i in range(10)])
results_pd = pd.DataFrame(results_table, index=guys, columns=girls)
display(results_pd)

status: 1, Optimal
objective: 7.0


,Jessica,Brittany,Simone,Shanley,Jacy,Amber,Paige,Kayla,Ashleigh,Coleysia
Ethan,0,0,0,0,0,1,0,0,0,0
Ryan,0,1,0,0,0,0,0,0,0,0
Wes,0,0,0,0,0,0,0,0,0,0
Chris S,0,0,1,0,0,0,0,0,0,0
Chris T,1,0,0,0,0,0,0,0,0,0
Dre,0,0,0,0,0,0,0,0,0,0
JJ,0,0,0,0,1,0,0,0,0,0
Adam,0,0,0,1,0,0,0,0,0,0
Dillan,0,0,0,0,0,0,0,0,0,1
Joey,0,0,0,0,0,0,0,0,0,0


In [ ]:
import cvxpy # to be experimented with

In [ ]:
from ortools.linear_solver import pywraplp # to be experimented with